In [49]:
try:
    spark.stop()
except NameError:
    pass

In [ ]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession

current_dir = os.getcwd() 
project_root = os.path.dirname(current_dir) 
dotenv_path = os.path.join(project_root, 'docker', '.env')

print(f"Aktuelles Arbeitsverzeichnis (getcwd): {project_root}")
print(f"Erwarteter Pfad zur .env-Datei: {dotenv_path}")

if not os.path.exists(dotenv_path):
    raise FileNotFoundError(f"FEHLER: Die .env-Datei wurde nicht im Pfad '{dotenv_path}' gefunden. Bitte stellen Sie sicher, dass Sie das Skript vom richtigen Verzeichnis aus starten.")
else:
    print("INFO: .env-Datei erfolgreich gefunden.")

load_dotenv(dotenv_path=dotenv_path)

MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")
MINIO_ACCESS_KEY = os.getenv("MINIO_ROOT_USER")
MINIO_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD")

if not all([MINIO_ENDPOINT ,MINIO_ACCESS_KEY, MINIO_SECRET_KEY]):
    raise ValueError("MinIO-Zugangsdaten nicht in der .env-Datei gefunden oder nicht geladen.")
    
spark = (
    SparkSession.builder
        .appName("Jupyter-Interactive-Session")
        .master("spark://spark-master:7077")  # <-- Entscheidend für Cluster-Verbindung
        
        # Konfiguration für JARs und MinIO-Treiber
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.688")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        
        # Konfiguration für MinIO-Zugangsdaten
        .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
        .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
        .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
        
        # Wichtige Netzwerkkonfiguration für Docker
        .config("spark.driver.host", "jupyter")
        .getOrCreate()
)


print("Spark Session wurde gestartet und erfolgreich für MinIO konfiguriert.")
print(f"Verbindung zu MinIO-Endpoint: {MINIO_ENDPOINT} mit User: {MINIO_ACCESS_KEY}")

Spark Session wurde gestartet und erfolgreich für MinIO konfiguriert.
Verbindung zu MinIO-Endpoint: http://localhost:9000 mit User: minioadmin


In [51]:
import json

config_file_path = os.path.join(project_root, 'config', 'data_config.json')

with open(config_file_path, 'r') as f:
    config = json.load(f)

print(f"Verarbeite Daten für: {config['dataSourceName']}")

Verarbeite Daten für: Heizkraftwerk


In [52]:
import sys

if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Projekt-Stammverzeichnis '{project_root}' zum sys.path hinzugefügt.")

from scripts import reader

bronze_df = reader.read_data(spark, config)
print("Bronze-Schema:")
bronze_df.printSchema()
bronze_df.show(5, truncate=False)

25/06/19 22:05:54 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://bronze/Data.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2737)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3569)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3612)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:55)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSour

Py4JJavaError: An error occurred while calling o274.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2737)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3569)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3612)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:172)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3716)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3667)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:557)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:366)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:777)
	at scala.collection.immutable.List.map(List.scala:247)
	at scala.collection.immutable.List.map(List.scala:79)
	at org.apache.spark.sql.executio